# 2018-2019 Player Classification

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def scrape_basketballreference(url):
    html = urlopen(url)
    soup = BeautifulSoup(html)
#     print(soup)
    soup.findAll('tr', limit=2)
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    return stats

In [3]:
totals_0 = scrape_basketballreference('https://www.basketball-reference.com/leagues/NBA_2019_totals.html')

In [4]:
totals = scrape_basketballreference('https://www.basketball-reference.com/leagues/NBA_2019_totals.html')
# per_game = scrape_basketballreference('https://www.basketball-reference.com/leagues/NBA_2019_per_game.html')
advanced = scrape_basketballreference('https://www.basketball-reference.com/leagues/NBA_2019_advanced.html')

In [5]:
advanced_0 = scrape_basketballreference('https://www.basketball-reference.com/leagues/NBA_2019_advanced.html')

In [6]:
# dropped some weird blank columns
advanced.drop(advanced.columns[[18,23]], axis=1)

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Álex Abrines,SG,25,OKC,31,588,6.3,.507,.809,.083,...,7.9,12.2,0.1,0.6,0.6,.053,-3.7,0.4,-3.3,-0.2
1,Quincy Acy,PF,28,PHO,10,123,2.9,.379,.833,.556,...,15.2,9.2,-0.1,0.0,-0.1,-0.022,-7.6,-0.5,-8.1,-0.2
2,Jaylen Adams,PG,22,ATL,34,428,7.6,.474,.673,.082,...,19.7,13.5,-0.1,0.2,0.1,.011,-3.8,-0.5,-4.3,-0.2
3,Steven Adams,C,25,OKC,80,2669,18.5,.591,.002,.361,...,12.6,16.4,5.1,4.0,9.1,.163,0.7,0.4,1.1,2.1
4,Bam Adebayo,C,21,MIA,82,1913,17.9,.623,.031,.465,...,17.1,15.8,3.4,3.4,6.8,.171,-0.4,2.2,1.8,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,Tyler Zeller,C,29,MEM,4,82,19.4,.640,.000,.643,...,10.0,21.8,0.2,0.1,0.3,.191,-2.3,-1.9,-4.3,0.0
730,Ante Žižić,C,22,CLE,59,1082,16.2,.590,.000,.399,...,13.6,18.2,1.7,0.3,2.0,.087,-1.1,-2.1,-3.2,-0.3
731,Ivica Zubac,C,21,TOT,59,1040,18.9,.604,.000,.332,...,13.9,20.3,1.9,1.3,3.2,.148,-0.2,-0.2,-0.5,0.4
732,Ivica Zubac,C,21,LAL,33,516,19.5,.633,.000,.342,...,12.9,20.5,1.1,0.6,1.8,.165,-0.3,0.1,-0.2,0.2


Find the columns that are the same name and determine if they're really the same

In [7]:
b = totals_0.columns.intersection(advanced_0.columns)
print(b)

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'MP'], dtype='object')


## Columns analysis:

Same: 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP' 

Groupby: 'Player', 'Pos'

Drop: 'MP_adv', 'Pos_adv', 'Age_adv', 'Pos_pg', 'Age_pg', 'G_pg', 'GS_pg'

Sum: 'MP', 

Average: 

Custom: 'GS_pg', 'MP_pg',
       'FG_pg', 'FGA_pg', 'FG%_pg', '3P_pg', '3PA_pg', '3P%_pg', '2P_pg',
       '2PA_pg', '2P%_pg', 'eFG%_pg', 'FT_pg', 'FTA_pg', 'FT%_pg', 'ORB_pg',
       'DRB_pg', 'TRB_pg', 'AST_pg', 'STL_pg', 'BLK_pg', 'TOV_pg', 'PF_pg',
       'PTS_pg' (sum up MP for all teams and divide by number of games in season)

In [8]:
tmp = pd.merge(totals, advanced, how='inner', left_on=['Player', 'Pos', 'Tm', 'Age', 'G', 'MP'], right_on=['Player','Pos', 'Tm', 'Age', 'G', 'MP'])

In [9]:
tmp.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', ' ', 'OWS', 'DWS', 'WS', 'WS/48', ' ', 'OBPM', 'DBPM', 'BPM',
       'VORP'],
      dtype='object')

In [10]:
tmp.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,Álex Abrines,SG,25,OKC,31,2,588,56,157,.357,...,,0.1,0.6,0.6,.053,,-3.7,0.4,-3.3,-0.2
1,Quincy Acy,PF,28,PHO,10,0,123,4,18,.222,...,,-0.1,0.0,-0.1,-0.022,,-7.6,-0.5,-8.1,-0.2
2,Jaylen Adams,PG,22,ATL,34,1,428,38,110,.345,...,,-0.1,0.2,0.1,.011,,-3.8,-0.5,-4.3,-0.2
3,Steven Adams,C,25,OKC,80,80,2669,481,809,.595,...,,5.1,4.0,9.1,.163,,0.7,0.4,1.1,2.1
4,Bam Adebayo,C,21,MIA,82,28,1913,280,486,.576,...,,3.4,3.4,6.8,.171,,-0.4,2.2,1.8,1.8
5,Deng Adel,SF,21,CLE,19,3,194,11,36,.306,...,,-0.2,0.0,-0.2,-0.054,,-6.0,-1.6,-7.5,-0.3
6,DeVaughn Akoon-Purcell,SG,25,DEN,7,0,22,3,10,.300,...,,-0.1,0.0,0.0,-0.051,,-7.9,2.1,-5.8,0.0
7,LaMarcus Aldridge,C,33,SAS,81,81,2687,684,1319,.519,...,,6.4,2.9,9.3,.167,,2.4,-0.6,1.8,2.6
8,Rawle Alkins,SG,21,CHI,10,1,120,13,39,.333,...,,-0.1,0.0,-0.1,-0.042,,-3.8,-3.5,-7.3,-0.2
9,Grayson Allen,SG,23,UTA,38,2,416,67,178,.376,...,,-0.4,0.4,0.0,.002,,-4.2,-2.1,-6.3,-0.5


In [11]:
# Find the columns where each value is null
empty_cols = [col for col in advanced.columns if advanced[col].empty]
# Drop these columns from the dataframe
advanced.drop(empty_cols,
        axis=1,
        inplace=True)

In [12]:
advanced.head(15)

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,Álex Abrines,SG,25,OKC,31,588,6.3,.507,.809,.083,...,,0.1,0.6,0.6,.053,,-3.7,0.4,-3.3,-0.2
1,Quincy Acy,PF,28,PHO,10,123,2.9,.379,.833,.556,...,,-0.1,0.0,-0.1,-0.022,,-7.6,-0.5,-8.1,-0.2
2,Jaylen Adams,PG,22,ATL,34,428,7.6,.474,.673,.082,...,,-0.1,0.2,0.1,.011,,-3.8,-0.5,-4.3,-0.2
3,Steven Adams,C,25,OKC,80,2669,18.5,.591,.002,.361,...,,5.1,4.0,9.1,.163,,0.7,0.4,1.1,2.1
4,Bam Adebayo,C,21,MIA,82,1913,17.9,.623,.031,.465,...,,3.4,3.4,6.8,.171,,-0.4,2.2,1.8,1.8
5,Deng Adel,SF,21,CLE,19,194,2.7,.424,.639,.111,...,,-0.2,0.0,-0.2,-0.054,,-6.0,-1.6,-7.5,-0.3
6,DeVaughn Akoon-Purcell,SG,25,DEN,7,22,8.2,.322,.400,.200,...,,-0.1,0.0,0.0,-0.051,,-7.9,2.1,-5.8,0.0
7,LaMarcus Aldridge,C,33,SAS,81,2687,22.9,.576,.032,.312,...,,6.4,2.9,9.3,.167,,2.4,-0.6,1.8,2.6
8,Rawle Alkins,SG,21,CHI,10,120,8.1,.418,.308,.308,...,,-0.1,0.0,-0.1,-0.042,,-3.8,-3.5,-7.3,-0.2
9,Grayson Allen,SG,23,UTA,38,416,7.5,.516,.556,.337,...,,-0.4,0.4,0.0,.002,,-4.2,-2.1,-6.3,-0.5


In [13]:
full = pd.merge(totals, advanced, how='inner', left_on=['Player', 'Pos', 'Tm', 'Age', 'G', 'MP'], right_on=['Player','Pos', 'Tm', 'Age', 'G', 'MP'])

In [14]:
full.shape

(1384, 51)

In [15]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [16]:
full[full.duplicated(keep=False)]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
22,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
23,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
24,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
25,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
26,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
694,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
695,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
696,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [17]:
full_dedup = full.drop_duplicates()

Inspect the data _x and _y and clean up (renaming, dropping blank columns)

Getting duplicates because they played on different teams in the same season. So, use the TOT row for players with multiple teams over the season.

In [18]:
# full_dedup.replace('TOT', 'TOR')

In [19]:
full_dedup = full_dedup.sort_values('Player')

In [20]:
full_dedup['Tm'].unique()

array(['ORL', 'IND', 'OKC', 'BOS', 'POR', 'BRK', 'CLE', 'TOT', 'SAC',
       'UTA', 'LAL', 'ATL', 'GSW', 'NYK', 'PHI', 'DET', 'NOP', 'MEM',
       'MIN', 'CHI', 'HOU', 'WAS', 'LAC', 'MIA', 'CHO', 'MIL', 'DEN',
       'SAS', 'TOR', 'DAL', 'PHO', None], dtype=object)

In [21]:
full_dedup.head(15)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
922,Aaron Gordon,PF,23,ORL,78,78,2633,470,1046,.449,...,,1.8,3.3,5.1,.093,,0.3,0.3,0.6,1.7
973,Aaron Holiday,PG,22,IND,50,0,646,105,262,.401,...,,0.1,0.8,0.9,.065,,-1.7,0.1,-1.6,0.1
1173,Abdel Nader,SF,25,OKC,61,1,694,91,215,.423,...,,0.0,0.9,0.9,.062,,-3.6,0.3,-3.2,-0.2
984,Al Horford,C,32,BOS,68,68,1973,387,723,.535,...,,4.5,2.9,7.5,.181,,3.3,1.9,5.1,3.6
12,Al-Farouq Aminu,PF,28,POR,81,81,2292,257,593,.433,...,,3.0,2.8,5.8,.121,,0.1,0.6,0.7,1.5
1356,Alan Williams,PF,26,BRK,5,0,26,8,13,.615,...,,0.1,0.1,0.2,.312,,7.6,4.2,11.8,0.1
775,Alec Burks,SG,27,CLE,34,24,979,136,340,.400,...,,0.6,0.3,0.9,.042,,-0.2,-1.2,-1.4,0.1
773,Alec Burks,SG,27,TOT,64,24,1375,192,474,.405,...,,0.7,0.8,1.5,.052,,-0.6,-0.7,-1.2,0.3
776,Alec Burks,SG,27,SAC,13,0,127,9,20,.450,...,,0.0,0.2,0.2,.062,,-3.9,2.8,-1.1,0.0
774,Alec Burks,SG,27,UTA,17,0,269,47,114,.412,...,,0.2,0.3,0.5,.084,,-0.4,-0.2,-0.6,0.1


To extract only the 'TOT' row for players who played for multiple teams in the season, we've already sorted by player name. Now, let's create a column that counts the number of times the player exists. Then, create a binary value column indicating whether or not 'Tm' is 'TOT'. Final step is to loop row by row using iloc[index,:] and delete row if no. occurences > 1 AND 'Tm' != 'TOT'

In [22]:
full_dedup['is_TOT'] = full_dedup.Tm.apply(lambda x:1 if x=="TOT" else 0)

In [23]:
full_dedup.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,is_TOT
922,Aaron Gordon,PF,23,ORL,78,78,2633,470,1046,.449,...,1.8,3.3,5.1,.093,,0.3,0.3,0.6,1.7,0
973,Aaron Holiday,PG,22,IND,50,0,646,105,262,.401,...,0.1,0.8,0.9,.065,,-1.7,0.1,-1.6,0.1,0
1173,Abdel Nader,SF,25,OKC,61,1,694,91,215,.423,...,0.0,0.9,0.9,.062,,-3.6,0.3,-3.2,-0.2,0
984,Al Horford,C,32,BOS,68,68,1973,387,723,.535,...,4.5,2.9,7.5,.181,,3.3,1.9,5.1,3.6,0
12,Al-Farouq Aminu,PF,28,POR,81,81,2292,257,593,.433,...,3.0,2.8,5.8,.121,,0.1,0.6,0.7,1.5,0
1356,Alan Williams,PF,26,BRK,5,0,26,8,13,.615,...,0.1,0.1,0.2,.312,,7.6,4.2,11.8,0.1,0
775,Alec Burks,SG,27,CLE,34,24,979,136,340,.400,...,0.6,0.3,0.9,.042,,-0.2,-1.2,-1.4,0.1,0
773,Alec Burks,SG,27,TOT,64,24,1375,192,474,.405,...,0.7,0.8,1.5,.052,,-0.6,-0.7,-1.2,0.3,1
776,Alec Burks,SG,27,SAC,13,0,127,9,20,.450,...,0.0,0.2,0.2,.062,,-3.9,2.8,-1.1,0.0,0
774,Alec Burks,SG,27,UTA,17,0,269,47,114,.412,...,0.2,0.3,0.5,.084,,-0.4,-0.2,-0.6,0.1,0


In [24]:
full_dedup.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', ' ', 'OWS', 'DWS', 'WS', 'WS/48', ' ', 'OBPM', 'DBPM', 'BPM',
       'VORP', 'is_TOT'],
      dtype='object')

In [25]:
full_dedup['Player_count'] = full_dedup.groupby('Player')['Player'].transform('size')

In [26]:
full_dedup.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', ' ', 'OWS', 'DWS', 'WS', 'WS/48', ' ', 'OBPM', 'DBPM', 'BPM',
       'VORP', 'is_TOT', 'Player_count'],
      dtype='object')

In [27]:
full_dedup.head(10) # column is created!

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,is_TOT,Player_count
922,Aaron Gordon,PF,23,ORL,78,78,2633,470,1046,.449,...,3.3,5.1,.093,,0.3,0.3,0.6,1.7,0,1.0
973,Aaron Holiday,PG,22,IND,50,0,646,105,262,.401,...,0.8,0.9,.065,,-1.7,0.1,-1.6,0.1,0,1.0
1173,Abdel Nader,SF,25,OKC,61,1,694,91,215,.423,...,0.9,0.9,.062,,-3.6,0.3,-3.2,-0.2,0,1.0
984,Al Horford,C,32,BOS,68,68,1973,387,723,.535,...,2.9,7.5,.181,,3.3,1.9,5.1,3.6,0,1.0
12,Al-Farouq Aminu,PF,28,POR,81,81,2292,257,593,.433,...,2.8,5.8,.121,,0.1,0.6,0.7,1.5,0,1.0
1356,Alan Williams,PF,26,BRK,5,0,26,8,13,.615,...,0.1,0.2,.312,,7.6,4.2,11.8,0.1,0,1.0
775,Alec Burks,SG,27,CLE,34,24,979,136,340,.400,...,0.3,0.9,.042,,-0.2,-1.2,-1.4,0.1,0,4.0
773,Alec Burks,SG,27,TOT,64,24,1375,192,474,.405,...,0.8,1.5,.052,,-0.6,-0.7,-1.2,0.3,1,4.0
776,Alec Burks,SG,27,SAC,13,0,127,9,20,.450,...,0.2,0.2,.062,,-3.9,2.8,-1.1,0.0,0,4.0
774,Alec Burks,SG,27,UTA,17,0,269,47,114,.412,...,0.3,0.5,.084,,-0.4,-0.2,-0.6,0.1,0,4.0


In [28]:
full_dedup.shape

(709, 53)

In [29]:
full_dedup_final = full_dedup[(full_dedup['is_TOT']==1)|((full_dedup['is_TOT']==0) & (full_dedup['Player_count']==1))]

In [30]:
full_dedup_final.shape

(530, 53)

In [31]:
full_dedup_final.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', ' ', 'OWS', 'DWS', 'WS', 'WS/48', ' ', 'OBPM', 'DBPM', 'BPM',
       'VORP', 'is_TOT', 'Player_count'],
      dtype='object')

In [32]:
full_dedup_final[full_dedup_final.columns[41]]

,,
922,,
973,,
1173,,
984,,
12,,
...,...,...
1199,,
1290,,
1227,,
0,,


In [33]:
full_dedup_final[full_dedup_final.columns[46]]

,,
922,,
973,,
1173,,
984,,
12,,
...,...,...
1199,,
1290,,
1227,,
0,,


In [34]:
full_dedup_final = full_dedup_final.drop(full_dedup_final.columns[[41,46]], axis=1)

In [35]:
full_dedup_final.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'is_TOT', 'Player_count'],
      dtype='object')

## PCA

We'll want to use PCA to reduce dimensionality since there are tons of features that we don't know necessarily which ones may be actually influential. It'll also be good to remove any features that may be highly correlated with another feature.